In [1]:
#lbbVIXAnalysis-2-2.ipynb
# Find the ending date of the current futures contract as of the date in the Trade Date column

# First solve for when the futures contract expires (or settles) 
#     - that is based on when it's relevant options contract (SPX) expires
#        - the SPX monthly options expire on the third Friday of every month 
#        - the expiration date of the third Friday changes if there is a holiday on that same date
#            - then it is the Thursday before the regular expiration date
#        - double check that the formula isn't fouled by leap years
#        - compute holidays and determine any conflicts with options expiration and adjust accordingly
# Once you know when the options end, then you can determine the end of the VX futures contract 
#        - spx options expiration - 30 days = settlement date for VX futures (its expiration date)
#        - the last trade date for the VX futures is NOT necessarily the same as the settlement date
#        - in addition to calculating the last trade date(LTD) for the current futures contract, 
#             -  also compute the LTD time, which changed throughout the years

In [2]:
# OPTION EXPIRATION DATE ***********

# CREATE THE FORMULA FROM TRADE DATE TO CALCULATE SPX OPTIONS EXPIRATION AND THEREFORE VX FUTURES EXPIRATION
#   - INCLUDES HOLIDAY 'NO' YEARS EFFECT ON OPTIONS EXPIRATION
#   - INCLUDES FOR FUTURES LTD AND XP DATE SPECIFICS
#   - INCLUDES FUTURES LTD DATE TIME SPECIFICS
#   - 

In [3]:
#lbbVIXAnalysis-2-2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

import datetime as dt
import os
os.getcwd()


'C:\\Users\\cinderella\\dataSci\\DataScience\\eliteDataScience\\07-CapstoneProject\\Final\\_projectNotebooks'

In [4]:
os.chdir('c:\\users\\cinderella\\datasci\\datascience\\elitedatascience\\07-cornerstoneproject\\Final')

In [5]:
df = pd.read_csv('abt_Initial.csv', index_col=0)

In [6]:
df.tail()

,Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest
33796,2020-08-17,Z (Dec 2020),28.06,28.10,27.31,27.60,27.375,-0.925,7541,0,18656
33797,2020-08-18,Z (Dec 2020),27.40,27.75,27.20,27.42,27.475,0.100,6174,0,19197
33798,2020-08-19,Z (Dec 2020),27.40,28.00,27.35,27.92,27.900,0.425,6049,0,20106
33799,2020-08-20,Z (Dec 2020),27.92,28.52,27.65,27.80,27.825,-0.075,6803,0,20760
33800,2020-08-21,Z (Dec 2020),27.77,28.45,27.66,28.24,28.175,0.350,5035,0,20592


In [7]:
df.count()

Trade Date       33792
Futures          33792
Open             33792
High             33792
Low              33792
Close            33792
Settle           33792
Change           33792
Total Volume     33792
EFP              33792
Open Interest    33792
dtype: int64

In [8]:
df.dtypes

Trade Date        object
Futures           object
Open             float64
High             float64
Low              float64
Close            float64
Settle           float64
Change           float64
Total Volume       int64
EFP                int64
Open Interest      int64
dtype: object

In [9]:
duplicateRowsDF = df[df.duplicated()]
duplicateRowsDF.shape
## Make sure this checks out with the last phase

(0, 11)

In [10]:
## don't need, no dups  duplicateRowsDF.head()  df = df.drop_duplicates()

In [11]:
df.shape


(33792, 11)

In [12]:
df.isnull().sum() #just to double check, should be null

Trade Date       0
Futures          0
Open             0
High             0
Low              0
Close            0
Settle           0
Change           0
Total Volume     0
EFP              0
Open Interest    0
dtype: int64

In [13]:
# make the trade date a dt
df['Trade Date'] = pd.to_datetime(df['Trade Date'])

In [14]:
df['calYear'] = df['Trade Date'].dt.year
df['calMth'] = df['Trade Date'].dt.month

In [15]:
df['opYear'] = np.where(df['calMth']==12, df['calYear'] + 1, df['calYear'])

In [16]:
df['opMth'] = np.where(df['calMth']==12, 1, df['calMth']+1)

In [17]:
df['opFOM'] = df.opYear.astype(str)+'-'+df.opMth.astype(str).str.zfill(2)+'-01'

In [18]:
# make df.opFOM into a datetime so we can get the day of the week number
df['opFOM'] = pd.to_datetime(df['opFOM'])

In [19]:
# now use datetime to get number of the day of the week, this daynum makes 
# 0 Mon leaving Saturday to be 5 and Sun 6
df['opDOW1st'] = df['opFOM'].dt.dayofweek
#df['opDOW']

In [20]:
#df['opDOW2nd'] = np.where(df['opDOW1st']< 5,  4 - df['opDOW1st'], 4)
df['opDOW2nd'] = np.where(df['opDOW1st'] < 5 , 4 - df['opDOW1st'], 
                          np.where(df['opDOW1st'] == 5, 6, 5))

In [21]:
df.opDOW2nd



0        4
1        4
2        4
3        4
4        4
5        4
6        4
7        2
8        2
9        2
10       2
11       2
12       2
13       2
14       2
15       2
16       2
17       2
18       2
19       2
20       2
21       2
22       2
23       2
24       2
25       2
26       2
27       2
28       6
29       6
30       6
31       6
32       6
33       6
34       6
35       6
37       6
38       6
39       6
40       6
41       6
42       6
43       6
44       6
45       6
46       6
47       6
48       6
49       6
50       3
51       3
52       3
53       3
54       3
55       3
56       3
57       3
58       3
59       3
60       3
61       3
62       1
63       1
64       1
65       1
66       1
67       1
68       1
69       1
70       5
71       5
72       5
73       5
74       5
75       5
76       5
77       5
78       5
79       5
80       5
81       5
82       5
83       5
84       5
85       5
86       5
87       5
88       5
89       5
90       5
91       2

In [22]:
d_numDay = {0 : 'Monday',
         1 : 'Tuesday',
         2 : 'Wednesday',
         3 : 'Thursday',
         4 : 'Friday',
         5 : 'Saturday',
         6 : 'Sunday'}


In [23]:
## 'No' years are years where the options expiration date and a holiday date (in this case always
## in March for 'easter', converged and the options expiration was moved up from Friday to 
## Thursday)

d_numNoYear = {2014 : 9,
            2019 : 9, 
            2022 : 9,
            2025 : 9,
            2030 : 9,
            2033 : 9,
            2041 : 9,
            2044 : 9,
            2049 : 9,
            2052 : 9,
            2055 : 9,
            2057 : 9,
            2060 : 9,
            2068 : 9,
            2071 : 9,
            2076 : 9}
            

In [24]:
# create a column that holds whether or not an spx option year is one of the NO Years,
# we need to understand this in order to add the appropriate number of days to eventually 
# derive the VX expiration

df['noYear'] = df['opYear'].map(d_numNoYear)
df['noYear'] = df['noYear'].fillna(0).astype(int)
df['noYear'].astype(int)

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
32       0
33       0
34       0
35       0
37       0
38       0
39       0
40       0
41       0
42       0
43       0
44       0
45       0
46       0
47       0
48       0
49       0
50       0
51       0
52       0
53       0
54       0
55       0
56       0
57       0
58       0
59       0
60       0
61       0
62       0
63       0
64       0
65       0
66       0
67       0
68       0
69       0
70       0
71       0
72       0
73       0
74       0
75       0
76       0
77       0
78       0
79       0
80       0
81       0
82       0
83       0
84       0
85       0
86       0
87       0
88       0
89       0
90       0
91       0

In [25]:
# Where there is a 'NO' year, AND whenever the month is March, we only add 13 to
# the opDOW2nd . otherwise, we add 14  to the spxDaysNum 
# these numbers represent the cummulative days we will add to the datetime var opFOM (options First of Month) 
# to calculate the opXP (date of SPX options expiration), upon which the VX expiration and front month
# are calculated from

df['spxDaysNum'] = np.where((df['noYear']== 9) & (df['calMth'] == 3), 
                            df['opDOW2nd'] + 13, df['opDOW2nd']+14)

In [26]:
df.dtypes


Trade Date       datetime64[ns]
Futures                  object
Open                    float64
High                    float64
Low                     float64
Close                   float64
Settle                  float64
Change                  float64
Total Volume              int64
EFP                       int64
Open Interest             int64
calYear                   int64
calMth                    int64
opYear                    int64
opMth                     int64
opFOM            datetime64[ns]
opDOW1st                  int64
opDOW2nd                  int64
noYear                    int32
spxDaysNum                int64
dtype: object

In [27]:

## Because opFOM is a datetime, and the number of days we calculated above for spxDaysNum is 
## an integer, in order to add the number of days to the datetime var opFOM to create 
## spxDateXP, we need to map the number of days we are adding to the datetime function 
## timedelta

# add spxDaysNum to the datetime object opFOM
df['opXP'] = df['opFOM'] + df['spxDaysNum'].map(dt.timedelta)

In [28]:
df['opXP_DOW'] = df.opXP.dt.dayofweek # the day of the week options expire (should always be Friday except for holiday years with month March)

In [29]:
df['weekday'] = df['opXP_DOW'].map(d_numDay) # the name of the day of the week options expire

In [30]:
# to find a certain value from a column use the following guidelines
#To select rows whose column value equals a scalar, some_value, use ==:
#   df.loc[df['column_name'] == some_value]
#To select rows whose column value is in an iterable, some_values, use isin:
#   df.loc[df['column_name'].isin(some_values)]
#Combine multiple conditions with &:
#   df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]

#To find all rows where spxDateXP_Day !- 4
df.loc[df['opXP_DOW']!= 4].head(5) # find all days the options expired that were not 4 (Friday)
## all of these dates are correct, the options closed a day early for the holidays

,Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest,calYear,calMth,opYear,opMth,opFOM,opDOW1st,opDOW2nd,noYear,spxDaysNum,opXP,opXP_DOW,weekday
7062,2014-03-03,H (Mar 2014),15.90,17.03,15.90,16.57,16.60,1.35,116949,0,140956,2014,3,2014,4,2014-04-01,1,3,9,16,2014-04-17,3,Thursday
7063,2014-03-04,H (Mar 2014),16.55,16.55,15.00,15.08,15.10,-1.50,81113,455,139349,2014,3,2014,4,2014-04-01,1,3,9,16,2014-04-17,3,Thursday
7064,2014-03-05,H (Mar 2014),15.10,15.40,14.95,15.13,15.15,0.05,61079,0,129192,2014,3,2014,4,2014-04-01,1,3,9,16,2014-04-17,3,Thursday
7065,2014-03-06,H (Mar 2014),15.13,15.25,14.90,15.02,15.05,-0.10,45964,599,125995,2014,3,2014,4,2014-04-01,1,3,9,16,2014-04-17,3,Thursday
7066,2014-03-07,H (Mar 2014),15.05,15.65,14.70,15.48,15.50,0.45,78296,0,118941,2014,3,2014,4,2014-04-01,1,3,9,16,2014-04-17,3,Thursday


In [31]:
# Finally we have the expiration date (and tested it) for spx options, (opXP)

In [32]:
## even though Trade Date is a datetime, because we are asking only for those rows that
## evaluate to a string, it returns the value, but it cannot compute datetime vars
## the same way, which is why we needed to map the int var spxdaysnum to the dt.timedelta
## functio04
df.loc[df['Trade Date'] == '2012-03-29'] ##find all rows where trade date is 3/28/2019

,Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest,calYear,calMth,opYear,opMth,opFOM,opDOW1st,opDOW2nd,noYear,spxDaysNum,opXP,opXP_DOW,weekday
9213,2012-03-29,J (Apr 2012),17.85,18.80,17.05,17.09,17.10,-0.55,39455,4187,115762,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday
12653,2012-03-29,K (May 2012),19.95,20.60,19.40,19.54,19.55,-0.15,21798,2016,62887,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday
15348,2012-03-29,M (Jun 2012),21.60,22.10,21.15,21.25,21.30,-0.10,6765,0,24346,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday
17919,2012-03-29,N (Jul 2012),22.95,23.47,22.40,22.60,22.60,-0.10,5274,37,22427,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday
21069,2012-03-29,Q (Aug 2012),24.13,24.75,23.80,24.10,24.10,0.00,3455,59,29038,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday
23621,2012-03-29,U (Sep 2012),25.32,25.85,25.05,25.30,25.30,0.10,3164,59,23776,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday
26253,2012-03-29,V (Oct 2012),26.33,26.75,25.95,26.25,26.25,-0.10,2927,22,10726,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday
29520,2012-03-29,X (Nov 2012),26.40,26.80,26.00,26.30,26.30,0.05,775,0,3154,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday
32190,2012-03-29,Z (Dec 2012),25.90,26.45,25.90,26.05,26.05,-0.20,229,0,311,2012,3,2012,4,2012-04-01,6,5,0,19,2012-04-20,4,Friday


In [33]:
## Settlement date and expiration date for futures contracts are considered to be the same thing
## Now that we know the expiration date of the OPTIONS contract (SPX), we can calculate the 
## expiration date of the FUTURES (M01) contract (in this case the VIX futures contract)

df['t01XP'] = df['opXP'] - pd.offsets.DateOffset(days=30)
df['t01XP'] = pd.to_datetime(df['t01XP'])

##**Last Trade Date may or may not be the same as the settlement date

In [34]:
df. head(5)


,Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest,calYear,calMth,opYear,opMth,opFOM,opDOW1st,opDOW2nd,noYear,spxDaysNum,opXP,opXP_DOW,weekday,t01XP
0,2004-10-21,F (Jan 2005),16.81,16.89,16.80,16.85,16.70,167.0,13,0,13,2004,10,2004,11,2004-11-01,0,4,0,18,2004-11-19,4,Friday,2004-10-20
1,2004-10-22,F (Jan 2005),16.67,17.00,16.63,17.00,16.94,2.4,74,0,87,2004,10,2004,11,2004-11-01,0,4,0,18,2004-11-19,4,Friday,2004-10-20
2,2004-10-25,F (Jan 2005),17.16,17.16,17.03,17.12,17.19,2.5,63,0,147,2004,10,2004,11,2004-11-01,0,4,0,18,2004-11-19,4,Friday,2004-10-20
3,2004-10-26,F (Jan 2005),17.92,17.92,16.99,17.00,16.85,-3.4,13,0,158,2004,10,2004,11,2004-11-01,0,4,0,18,2004-11-19,4,Friday,2004-10-20
4,2004-10-27,F (Jan 2005),16.85,17.03,16.81,16.81,16.76,-0.9,60,0,177,2004,10,2004,11,2004-11-01,0,4,0,18,2004-11-19,4,Friday,2004-10-20


In [35]:
## Since last trade date(LTD), may not be the same as expiring date (XP) compute the LTD

## the last trade date for the SPX expiring options, if expiration is after Oct 22, 2014,
##     is the SAME as the settlement date
## for any other last trade dates (the ones before October 22, 2014), the last trade date 
##     is ONE DAY BEFORE the settlement date
 
df['LTD'] = np.where(df['t01XP'] > '2014-10-22', df['t01XP'], df['t01XP']- pd.offsets.DateOffset(days=1))

In [36]:
df.tail(5)

,Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest,calYear,calMth,opYear,opMth,opFOM,opDOW1st,opDOW2nd,noYear,spxDaysNum,opXP,opXP_DOW,weekday,t01XP,LTD
33796,2020-08-17,Z (Dec 2020),28.06,28.10,27.31,27.60,27.375,-0.925,7541,0,18656,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19
33797,2020-08-18,Z (Dec 2020),27.40,27.75,27.20,27.42,27.475,0.100,6174,0,19197,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19
33798,2020-08-19,Z (Dec 2020),27.40,28.00,27.35,27.92,27.900,0.425,6049,0,20106,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19
33799,2020-08-20,Z (Dec 2020),27.92,28.52,27.65,27.80,27.825,-0.075,6803,0,20760,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19
33800,2020-08-21,Z (Dec 2020),27.77,28.45,27.66,28.24,28.175,0.350,5035,0,20592,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19


In [37]:
df['dt01'] = "2014-10-22" #if expiration date is after this date, then last trade date is the same as the settlement date

In [38]:
df['dt02'] = "2015-10-21" #the time of expiration changed as well

In [39]:
df['dt02'] = pd.to_datetime(df['dt02'])

In [40]:
df['dt01'] = pd.to_datetime(df['dt01'])

In [41]:
df['LTD-TIME'] = np.where(df['LTD'] < df['dt01'],"3:15 PM CT", np.where(df['LTD'] >= df['dt02'], "8:00 AM CT", "7:30 AM CT"))

In [42]:
df.tail()

,Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest,calYear,calMth,opYear,opMth,opFOM,opDOW1st,opDOW2nd,noYear,spxDaysNum,opXP,opXP_DOW,weekday,t01XP,LTD,dt01,dt02,LTD-TIME
33796,2020-08-17,Z (Dec 2020),28.06,28.10,27.31,27.60,27.375,-0.925,7541,0,18656,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT
33797,2020-08-18,Z (Dec 2020),27.40,27.75,27.20,27.42,27.475,0.100,6174,0,19197,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT
33798,2020-08-19,Z (Dec 2020),27.40,28.00,27.35,27.92,27.900,0.425,6049,0,20106,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT
33799,2020-08-20,Z (Dec 2020),27.92,28.52,27.65,27.80,27.825,-0.075,6803,0,20760,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT
33800,2020-08-21,Z (Dec 2020),27.77,28.45,27.66,28.24,28.175,0.350,5035,0,20592,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT


In [43]:
df.shape

(33792, 28)

In [44]:
df.to_csv('abt-2.csv')